In [3]:

import os
import time
import oss2
import json

import cv2 as cv
import numpy as np
from tqdm import tqdm
from sko.GA import GA

from aliyunsdkcore.client import AcsClient
from aliyunsdkcore.acs_exception.exceptions import ClientException
from aliyunsdkcore.acs_exception.exceptions import ServerException
from aliyunsdkfacebody.request.v20191230.CompareFaceRequest import CompareFaceRequest


In [12]:
import oss2
import aliyunsdkcore
import aliyunsdkfacebody
aliyunsdkcore.__version__, aliyunsdkfacebody.__version__, oss2.__version__


('2.13.11', '1.2.0', '2.13.0')

In [4]:

access_key_id = os.getenv('OSS_TEST_ACCESS_KEY_ID', '')
access_key_secret = os.getenv('OSS_TEST_ACCESS_KEY_SECRET', '')
bucket_name = os.getenv('OSS_TEST_BUCKET', 'ivan-bucket-out-002')
endpoint = os.getenv('OSS_TEST_ENDPOINT', 'oss-cn-shanghai.aliyuncs.com')

bucket = oss2.Bucket(oss2.Auth(access_key_id, access_key_secret), endpoint, bucket_name)

client = AcsClient(access_key_id, access_key_secret, 'cn-shanghai')


In [5]:

#
def _f1(imgA=None, imgB=None, Rect=None):
    while True:
        try:
            time.sleep(0.68)
            if imgA:
                oss2.resumable_upload(bucket, 'imgA.jpg', imgA)
            if imgB:
                oss2.resumable_upload(bucket, 'imgB.jpg', imgB)

            request = CompareFaceRequest()
            request.set_accept_format('json')

            request.set_ImageURLA(f"https://{bucket_name}.{endpoint}/imgA.jpg")
            request.set_ImageURLB(f"https://{bucket_name}.{endpoint}/imgB.jpg")

            response = client.do_action_with_exception(request)
            
            if Rect:
                return json.loads(response)["Data"]["Confidence"], json.loads(response)["Data"]["RectAList"]
            else:
                return json.loads(response)["Data"]["Confidence"]
        except Exception as e:
            print(e)

#
def _f2(imgA, imgB):
    imgA, imgB = cv.imread(imgA), cv.imread(imgB)
    return np.sqrt(np.sum(np.square(imgA - imgB)))


In [6]:
%time _f1(imgA='../Data/Z0.jpg', imgB='../Data/Z0.jpg')

CPU times: user 16.4 ms, sys: 8.96 ms, total: 25.4 ms
Wall time: 1.22 s


100.0

In [5]:
%time _f1(imgA='../Data/Z0.jpg', Rect=1)

CPU times: user 5.92 ms, sys: 1.93 ms, total: 7.85 ms
Wall time: 945 ms


(100.0, [19, 22, 77, 89])

In [6]:
%time _f1(imgA='../Data/Z0.jpg')

CPU times: user 5.89 ms, sys: 2.38 ms, total: 8.28 ms
Wall time: 987 ms


100.0

In [7]:
%time _f1(imgA='../Data/Z0.jpg', imgB='../Data/Z1.jpg')

CPU times: user 8.04 ms, sys: 2.37 ms, total: 10.4 ms
Wall time: 1.02 s


45.32559585571289

In [8]:
%time _f1(imgA='../Data/bad.png', imgB='../Data/bad.png')

CPU times: user 8.48 ms, sys: 3.13 ms, total: 11.6 ms
Wall time: 1.02 s


0.0

In [9]:
%time _f2(imgA='../Data/Z0.jpg', imgB='../Data/Z1.jpg')

CPU times: user 1.32 ms, sys: 1.57 ms, total: 2.89 ms
Wall time: 1.4 ms


768.1659976853962

In [10]:
%time _f2(imgA='../Data/Z0.jpg', imgB='../Data/Z0.jpg')

CPU times: user 1.12 ms, sys: 848 µs, total: 1.97 ms
Wall time: 926 µs


0.0

In [11]:

pNum = "T000001"

proj = ".."
pimg = f"{proj}/Data/images"
pmdl = f"{proj}/Model"
pout = f"{proj}/Out/{pNum}"

os.system(f"rm -rf {pout}/temp*")
os.system(f"rm -rf {pout}/images/*")


0

In [12]:
# ! rm -rf ../Out/T000001/images/*

In [13]:

st, ed = 1, 1
N, Cmin, Cmax = 112, 0, 255
S = N**2*3


In [14]:

RUN = 200
BIG = 25
CUT = 50

for i in tqdm(range(st, ed+1), desc="All"):
    mscore = 0
    pi = f"{pimg}/"+f"00000{i}"[-5:]+".jpg"
    op = f"{pout}/images/"+f"00000{i}"[-5:]+".jpg"
    tp = f"{pout}/temp.jpg"
    
    mescore, [x, y, w, h] = _f1(imgA=pi, imgB=pi, Rect=1)
    min_y, max_y, min_x, max_x = y, y+h, x, x+w
    print(mescore, min_y, max_y, min_x, max_x)
    if mescore == 0:# or os.path.exists(op):
        print(pi, "%.6f"%mescore)
        continue
    
    imgb = cv.imread(pi)
    score = {}
    scmin = [99999]
    
    # 
    def _function(X):
        _X = X.tostring()
        
        if _X in score:
            return score[_X]
        else:
            imgo1 = imgb.copy()
            # imgo2 = imgb.copy()
            imgo1 = imgo1 + X.reshape((N, N, 3)) 
            # imgo2[min_y:max_y, min_x:max_x] = imgo1[min_y:max_y, min_x:max_x]
            cv.imwrite(tp, imgo1)
            
#             _s1 = _f1(imgB=tp)
            _s1 = 1
            _s2 = _f2(imgA=pi, imgB=tp)

#             _s1 = CUT if _s1 <= CUT else _s1
#             _s2 = _s2
            _s_ = (2*_s1*_s2)/(_s1+_s2)
            
            if _s_ < min(scmin):
                scmin.append(_s_)
            score[_X] = _s_
            
            print("Min:%.2f, Avg:%.6f, S1:%.6f, S2:%.6f, SC:%.6f" % (min(scmin), np.mean(np.abs(X)), _s1, _s2, _s_))
            return _s_
    
    try:
        _lb = np.array([max(_-BIG, Cmin)-_ for _ in imgb.ravel()])
        _ub = np.array([min(_+BIG, Cmax)-_ for _ in imgb.ravel()])
        # print(_lb, _ub)
        
        """
        入参|默认值|意义
        func|-|目标函数
        n_dim|-|目标函数的维度
        size_pop|50|种群规模
        max_iter|200|最大迭代次数
        prob_mut|0.001|变异概率
        lb|-1|每个参数的最小值
        ub|1|每个参数的最大值
        constraint_eq|空元组|线性约束
        constraint_ueq|空元组|非线性约束
        precision|1e-7|精准度，int/float或者它们组成的列表
        """
        # from sko.DE import DE as GA
        # _function.is_vector = True
        ga = GA(
            func=_function,
            n_dim=S,
            size_pop=50,
            max_iter=RUN,
            prob_mut=0.1,
            lb=_lb, 
            ub=_ub,
            precision=[1 for _ in range(S)]
        )
        best_x, best_y = ga.run()
        # ga.Chrom = np.random.randint(0, 1, size=(1, S))
    except Exception as e:
        print(e)

    X = best_x
    imgb = cv.imread(pi)
    imgo1 = imgb.copy()
    # imgo2 = imgb.copy()
    imgo1 = imgo1 + X.reshape((N, N, 3)) 
    # imgo2[min_y:max_y, min_x:max_x] = imgo1[min_y:max_y, min_x:max_x]
    cv.imwrite(op, imgo1)
    print(">>>", pi, "%.6f"%_f1(imgA=f"{pi}", imgB=f"{op}"), "%.6f"%_f2(imgA=f"{pi}", imgB=f"{op}"))




All:   0%|          | 0/1 [00:00<?, ?it/s]

99.99996185302734 17 111 19 103
Min:2.00, Avg:14.994287, S1:1.000000, S2:1732.403244, SC:1.998846
Min:2.00, Avg:15.001568, S1:1.000000, S2:1730.945695, SC:1.998845
Min:2.00, Avg:15.080623, S1:1.000000, S2:1740.088791, SC:1.998851
Min:2.00, Avg:15.038584, S1:1.000000, S2:1735.184140, SC:1.998848
Min:2.00, Avg:14.931122, S1:1.000000, S2:1728.098666, SC:1.998843
Min:2.00, Avg:15.003720, S1:1.000000, S2:1728.327226, SC:1.998843
Min:2.00, Avg:15.029310, S1:1.000000, S2:1730.248537, SC:1.998845
Min:2.00, Avg:15.047300, S1:1.000000, S2:1725.822702, SC:1.998842
Min:2.00, Avg:14.980708, S1:1.000000, S2:1721.767696, SC:1.998839
Min:2.00, Avg:15.022561, S1:1.000000, S2:1733.369262, SC:1.998847
Min:2.00, Avg:14.984641, S1:1.000000, S2:1730.115314, SC:1.998845
Min:2.00, Avg:15.006564, S1:1.000000, S2:1726.728989, SC:1.998842
Min:2.00, Avg:14.988520, S1:1.000000, S2:1726.523965, SC:1.998842
Min:2.00, Avg:14.952168, S1:1.000000, S2:1734.782984, SC:1.998848
Min:2.00, Avg:14.941778, S1:1.000000, S2:173

Min:2.00, Avg:14.940636, S1:1.000000, S2:1723.668762, SC:1.998840
Min:2.00, Avg:14.935002, S1:1.000000, S2:1728.231466, SC:1.998843
Min:2.00, Avg:14.974835, S1:1.000000, S2:1729.765013, SC:1.998844
Min:2.00, Avg:14.969335, S1:1.000000, S2:1725.665379, SC:1.998842
Min:2.00, Avg:15.014934, S1:1.000000, S2:1731.177923, SC:1.998845
Min:2.00, Avg:15.021312, S1:1.000000, S2:1725.052173, SC:1.998841
Min:2.00, Avg:14.952354, S1:1.000000, S2:1728.355866, SC:1.998844
Min:2.00, Avg:14.999389, S1:1.000000, S2:1730.940785, SC:1.998845
Min:2.00, Avg:15.069143, S1:1.000000, S2:1728.149588, SC:1.998843
Min:2.00, Avg:15.019877, S1:1.000000, S2:1730.794904, SC:1.998845
Min:2.00, Avg:14.989371, S1:1.000000, S2:1735.467660, SC:1.998848
Min:2.00, Avg:14.983392, S1:1.000000, S2:1731.186010, SC:1.998845
Min:2.00, Avg:15.031542, S1:1.000000, S2:1727.523082, SC:1.998843
Min:2.00, Avg:15.069701, S1:1.000000, S2:1732.274228, SC:1.998846
Min:2.00, Avg:14.963462, S1:1.000000, S2:1734.441697, SC:1.998848
Min:2.00, 

Min:2.00, Avg:15.008477, S1:1.000000, S2:1729.406256, SC:1.998844
Min:2.00, Avg:15.033110, S1:1.000000, S2:1734.275353, SC:1.998847
Min:2.00, Avg:15.063962, S1:1.000000, S2:1729.834385, SC:1.998844
Min:2.00, Avg:15.015758, S1:1.000000, S2:1734.335031, SC:1.998847
Min:2.00, Avg:15.004092, S1:1.000000, S2:1730.540956, SC:1.998845
Min:2.00, Avg:15.065529, S1:1.000000, S2:1735.291906, SC:1.998848
Min:2.00, Avg:15.000824, S1:1.000000, S2:1727.800336, SC:1.998843
Min:2.00, Avg:15.071933, S1:1.000000, S2:1735.864050, SC:1.998848
Min:2.00, Avg:14.972577, S1:1.000000, S2:1726.276629, SC:1.998842
Min:2.00, Avg:14.980203, S1:1.000000, S2:1723.943735, SC:1.998841
Min:2.00, Avg:15.038265, S1:1.000000, S2:1725.100287, SC:1.998841
Min:2.00, Avg:15.057478, S1:1.000000, S2:1734.694786, SC:1.998848
Min:2.00, Avg:15.032579, S1:1.000000, S2:1729.948843, SC:1.998845
Min:2.00, Avg:15.040205, S1:1.000000, S2:1729.959537, SC:1.998845
Min:2.00, Avg:15.045812, S1:1.000000, S2:1730.596140, SC:1.998845
Min:2.00, 

Min:2.00, Avg:15.004703, S1:1.000000, S2:1728.496167, SC:1.998844
Min:2.00, Avg:15.022295, S1:1.000000, S2:1734.502234, SC:1.998848
Min:2.00, Avg:14.959928, S1:1.000000, S2:1723.100694, SC:1.998840
Min:2.00, Avg:15.049054, S1:1.000000, S2:1734.749838, SC:1.998848
Min:2.00, Avg:14.998990, S1:1.000000, S2:1729.053209, SC:1.998844
Min:2.00, Avg:15.055352, S1:1.000000, S2:1725.388362, SC:1.998842
Min:2.00, Avg:15.043447, S1:1.000000, S2:1733.778244, SC:1.998847
Min:2.00, Avg:14.992958, S1:1.000000, S2:1725.863262, SC:1.998842
Min:2.00, Avg:14.995908, S1:1.000000, S2:1725.568312, SC:1.998842
Min:2.00, Avg:15.004757, S1:1.000000, S2:1732.642779, SC:1.998846
Min:2.00, Avg:14.982541, S1:1.000000, S2:1728.188647, SC:1.998843
Min:2.00, Avg:15.006962, S1:1.000000, S2:1728.268498, SC:1.998843
Min:2.00, Avg:15.011958, S1:1.000000, S2:1729.168008, SC:1.998844
Min:2.00, Avg:15.040737, S1:1.000000, S2:1732.660382, SC:1.998846
Min:2.00, Avg:15.007626, S1:1.000000, S2:1730.628787, SC:1.998845
Min:2.00, 

...

Min:2.00, Avg:14.973693, S1:1.000000, S2:1729.469283, SC:1.998844
Min:2.00, Avg:14.946535, S1:1.000000, S2:1724.706642, SC:1.998841
Min:2.00, Avg:14.940396, S1:1.000000, S2:1726.907062, SC:1.998843
Min:2.00, Avg:14.990088, S1:1.000000, S2:1731.739299, SC:1.998846
Min:2.00, Avg:14.971221, S1:1.000000, S2:1725.823572, SC:1.998842
Min:2.00, Avg:14.940024, S1:1.000000, S2:1721.547850, SC:1.998839
Min:2.00, Avg:15.000957, S1:1.000000, S2:1723.387942, SC:1.998840
Min:2.00, Avg:14.949139, S1:1.000000, S2:1732.456349, SC:1.998846
Min:2.00, Avg:14.992134, S1:1.000000, S2:1730.374815, SC:1.998845
Min:2.00, Avg:15.019239, S1:1.000000, S2:1732.329645, SC:1.998846
Min:2.00, Avg:15.087426, S1:1.000000, S2:1728.317101, SC:1.998843
Min:2.00, Avg:15.001594, S1:1.000000, S2:1725.050144, SC:1.998841
Min:2.00, Avg:14.960645, S1:1.000000, S2:1727.421199, SC:1.998843
Min:2.00, Avg:15.015811, S1:1.000000, S2:1734.324941, SC:1.998847
Min:2.00, Avg:14.965163, S1:1.000000, S2:1723.201961, SC:1.998840
Min:2.00, 

Min:2.00, Avg:14.999947, S1:1.000000, S2:1728.466372, SC:1.998844
Min:2.00, Avg:14.967740, S1:1.000000, S2:1725.743028, SC:1.998842
Min:2.00, Avg:15.008450, S1:1.000000, S2:1732.138851, SC:1.998846
Min:2.00, Avg:14.984906, S1:1.000000, S2:1726.707850, SC:1.998842
Min:2.00, Avg:14.908243, S1:1.000000, S2:1727.415989, SC:1.998843
Min:2.00, Avg:15.018681, S1:1.000000, S2:1728.989011, SC:1.998844
Min:2.00, Avg:14.968458, S1:1.000000, S2:1726.891716, SC:1.998843
Min:2.00, Avg:14.887064, S1:1.000000, S2:1714.764415, SC:1.998834
Min:2.00, Avg:15.035714, S1:1.000000, S2:1735.082707, SC:1.998848
Min:2.00, Avg:14.921928, S1:1.000000, S2:1729.344963, SC:1.998844
Min:2.00, Avg:14.946482, S1:1.000000, S2:1727.037927, SC:1.998843
Min:2.00, Avg:14.980894, S1:1.000000, S2:1731.781741, SC:1.998846
Min:2.00, Avg:14.973453, S1:1.000000, S2:1732.904498, SC:1.998847
Min:2.00, Avg:14.970610, S1:1.000000, S2:1730.319335, SC:1.998845
Min:2.00, Avg:14.964498, S1:1.000000, S2:1742.043340, SC:1.998853
Min:2.00, 

Min:2.00, Avg:14.970132, S1:1.000000, S2:1723.887177, SC:1.998841
Min:2.00, Avg:15.073953, S1:1.000000, S2:1727.382992, SC:1.998843
Min:2.00, Avg:15.000903, S1:1.000000, S2:1735.894006, SC:1.998849
Min:2.00, Avg:14.984906, S1:1.000000, S2:1731.376331, SC:1.998846
Min:2.00, Avg:14.929634, S1:1.000000, S2:1725.680735, SC:1.998842
Min:2.00, Avg:14.982010, S1:1.000000, S2:1727.524819, SC:1.998843
Min:2.00, Avg:14.974330, S1:1.000000, S2:1728.416038, SC:1.998844
Min:2.00, Avg:14.995057, S1:1.000000, S2:1723.709662, SC:1.998840
Min:2.00, Avg:14.944303, S1:1.000000, S2:1718.754491, SC:1.998837
Min:2.00, Avg:14.957164, S1:1.000000, S2:1725.326056, SC:1.998841
Min:2.00, Avg:15.027317, S1:1.000000, S2:1725.153327, SC:1.998841
Min:2.00, Avg:14.996120, S1:1.000000, S2:1728.433973, SC:1.998844
Min:2.00, Avg:15.026307, S1:1.000000, S2:1725.384595, SC:1.998842
Min:2.00, Avg:15.001701, S1:1.000000, S2:1732.350715, SC:1.998846
Min:2.00, Avg:15.102253, S1:1.000000, S2:1724.612710, SC:1.998841
Min:2.00, 

Min:2.00, Avg:14.988972, S1:1.000000, S2:1725.317362, SC:1.998841
Min:2.00, Avg:14.953391, S1:1.000000, S2:1729.209068, SC:1.998844
Min:2.00, Avg:15.063430, S1:1.000000, S2:1733.992791, SC:1.998847
Min:2.00, Avg:14.968086, S1:1.000000, S2:1732.705976, SC:1.998846
Min:2.00, Avg:15.018628, S1:1.000000, S2:1737.325531, SC:1.998849
Min:2.00, Avg:15.085592, S1:1.000000, S2:1725.985516, SC:1.998842
Min:2.00, Avg:14.949857, S1:1.000000, S2:1727.798889, SC:1.998843
Min:2.00, Avg:14.976802, S1:1.000000, S2:1725.886149, SC:1.998842
Min:2.00, Avg:14.946827, S1:1.000000, S2:1737.753147, SC:1.998850
Min:2.00, Avg:15.000345, S1:1.000000, S2:1736.624312, SC:1.998849
Min:2.00, Avg:14.985039, S1:1.000000, S2:1720.186908, SC:1.998838
Min:2.00, Avg:15.035236, S1:1.000000, S2:1730.220217, SC:1.998845
Min:2.00, Avg:14.982807, S1:1.000000, S2:1730.868857, SC:1.998845
Min:2.00, Avg:15.003428, S1:1.000000, S2:1721.688706, SC:1.998839
Min:2.00, Avg:15.005820, S1:1.000000, S2:1727.554920, SC:1.998843
Min:2.00, 

KeyboardInterrupt: 

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt

Y_history = pd.DataFrame(ga.all_history_Y)
fig, ax = plt.subplots(2, 1)
ax[0].plot(Y_history.index, Y_history.values, '.', color='red')
Y_history.min(axis=1).cummin().plot(kind='line')
plt.show()


In [ ]:

# st, el = 1, 712

# savl = []
# for i in tqdm(range(st, el+1)):
#     pi = f"{pimg}/"+f"00000{i}"[-5:]+".jpg"
#     op = f"{pout}/images/"+f"00000{i}"[-5:]+".jpg"
    
#     if not os.path.exists(op):
#         img = cv.imread(pi)
#         cv.imwrite(op, img)
#         savl.append(1)
# print(len(savl))


In [ ]:
# ! sh run.sh

In [ ]:
imgb = cv.imread(pi)